#pose잡기
import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    enable_segmentation=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # BGR -> RGB 변환
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Pose 처리
    results = pose.process(rgb)

    # 랜드마크 그리기
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS
        )

    cv2.imshow('Pose', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [1]:
#pose 감지
#왼팔 들기/오른팔 들기/양팔 들기/그외 로 나누어서 감지한다
import cv2
import mediapipe as mp

# MediaPipe Pose 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

# 랜드마크 인덱스 (MediaPipe Pose)
LEFT_SHOULDER = 11
RIGHT_SHOULDER = 12
LEFT_WRIST = 15
RIGHT_WRIST = 16

def is_arm_raised(shoulder_y, wrist_y, threshold=0.05):
    """
    어깨 y좌표보다 손목 y좌표가 특정 기준만큼 더 위(작은 값)이면 '팔을 들었다'고 판단.
    - Mediapipe Pose의 y값은 화면 상단이 0, 하단이 1. 작을수록 더 위쪽.
    - threshold는 오차 보정용(예: 0.05)
    """
    # 손목이 어깨보다 좀 더 위에 있으면 True
    return wrist_y < (shoulder_y - threshold)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽어올 수 없습니다.")
        break

    # BGR -> RGB
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Pose 추론
    results = pose.process(rgb)

    # 동작판별 결과 메세지
    action_text = ""

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # 왼/오른 어깨, 손목의 y좌표
        left_shoulder_y = landmarks[LEFT_SHOULDER].y
        right_shoulder_y = landmarks[RIGHT_SHOULDER].y
        left_wrist_y = landmarks[LEFT_WRIST].y
        right_wrist_y = landmarks[RIGHT_WRIST].y

        # 왼/오른팔 들었는지 판별
        left_arm_up = is_arm_raised(left_shoulder_y, left_wrist_y, threshold=0.05)
        right_arm_up = is_arm_raised(right_shoulder_y, right_wrist_y, threshold=0.05)

        if left_arm_up and right_arm_up:
            action_text = "both arms up"
        elif left_arm_up:
            action_text = "left arm up"
        elif right_arm_up:
            action_text = "right arm up"
        else:
            action_text = "do nothing"

        # 랜드마크 + 연결선 그리기
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS
        )

    # 동작 결과 표시
    cv2.putText(frame, action_text, (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    cv2.imshow("Pose Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 